In [8]:
search_for = 67
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.009759187698364258
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 20456441
type: [1, 1, 1, 1, 67] 67
cases of this type: 20151121
10000 3.1986585573730006
20000 7.369372800781191
30000 8.256502681823545
40000 9.561671549375825
50000 9.096772033395787
60000 9.668209203481913
70000 8.676573362976837
80000 9.557299825118243
90000 8.872552840436438
100000 8.805995262464954
110000 8.131932646849375
120000 8.70315228173869
130000 8.52775481405455
140000 9.027110250395438
150000 1.903058036337349
160000 1.9117344432792929
170000 8.633128683344244
180000 8.129370523909026
190000 8.829376699255539
200000 8.088432289046345
210000 8.270199883531994
220000 8.768627833205574
230000 9.74626480868568
240000 8.901652859801697
250000 8.284220469836566
260000 8.42853805718714
270000 8.645528414346568
280000 8.687764117736823
290000 7.861946334083497
300000 4.297439982125838
310000 3.636636420996421
320000 4.2586

3110000 5.416049707083265
3120000 4.096345400072727
3130000 4.741146329156061
3140000 4.3441953247443434
3150000 2.9411970130069123
3160000 1.3896652808206478
3170000 1.6989114104529421
3180000 4.934233242853947
3190000 4.733133176387065
3200000 4.934072879808267
3210000 5.18224865240954
3220000 4.906027161326255
3230000 5.250343250105382
3240000 5.501568410642107
3250000 5.303524605049193
3260000 2.9118226896676025
3270000 2.5803492721116474
3280000 4.183162878919561
3290000 6.001156255086502
3300000 4.97645164980989
3310000 5.178557778678218
3320000 5.005632254191696
3330000 4.322873311123391
3340000 4.031176761621091
3350000 5.176964848448277
3360000 2.638653307030062
3370000 2.756873128215876
3380000 4.76083910143288
3390000 5.109955082751711
3400000 5.6863972324554455
3410000 5.2374123637883665
3420000 4.873158329790427
3430000 4.655169392671738
3440000 4.942546832190354
3450000 3.8026639184811377
3460000 1.4276468678875367
3470000 1.1473375147769849
3480000 5.3288397414399125
349

6250000 4.00745488449241
6260000 3.8295090321318903
6270000 4.007704093347649
6280000 4.350810833802217
6290000 5.233194453426183
6300000 4.5465746382239
6310000 3.6700892944093013
6320000 4.447063565670014
6330000 4.262208120026078
6340000 4.934502296143082
6350000 4.097377085140864
6360000 4.424057830261542
6370000 4.452774022782895
6380000 3.2666157497059105
6390000 3.055198115503185
6400000 3.9298079446513117
6410000 2.7013019830185176
6420000 2.1232424940956167
6430000 3.680659223068138
6440000 3.2363483680069245
6450000 4.618834321719349
6460000 3.1405197135084664
6470000 1.250286212485969
6480000 2.037158459666497
6490000 4.353303674579561
6500000 4.276098254882037
6510000 3.9653686238626906
6520000 2.713682701962438
6530000 2.5282184457805155
6540000 4.510466604875114
6550000 4.102660966465082
6560000 4.127521341420829
6570000 4.526981444936129
6580000 4.320097336171315
6590000 4.310314737217645
6600000 3.9389248992779256
6610000 3.549347722463541
6620000 4.858294439774731
6630

9360000 0.8519433069570065
9370000 0.8571644787300693
9380000 0.858547257752061
9390000 0.7407882530321744
9400000 0.8537709362510046
9410000 0.5448229334041874
9420000 0.7679380620742904
9430000 0.5584344740344682
9440000 0.8413540624276399
9450000 0.58230097204062
9460000 0.6465552855932447
9470000 0.5353549637136658
9480000 0.6125540400991307
9490000 0.6559959741426176
9500000 0.5857154661762118
9510000 0.5024925387844112
9520000 0.4706676708123088
9530000 0.48624442520910505
9540000 0.7467875749815769
9550000 0.8461726329446699
9560000 0.8569519468019804
9570000 0.5007416483664976
9580000 0.47987636145091717
9590000 0.8342523092710773
9600000 0.8401089528226258
9610000 0.8009118588117428
9620000 0.84656494508183
9630000 0.8344124977951646
9640000 0.8372375437942943
9650000 0.8081526845827303
9660000 0.639949866167552
9670000 0.7119856638388766
9680000 0.8418562828112642
9690000 0.8302330556371477
9700000 0.8400627883575426
9710000 0.824739388217926
9720000 0.8356281181950702
973000

12340000 3.156283259674185
12350000 2.2407408987204827
12360000 1.9669661761259039
12370000 1.9341572285371356
12380000 1.983607540470143
12390000 2.1551446877155636
12400000 2.6675942753647766
12410000 2.704837178022325
12420000 2.117570070284764
12430000 2.3565678320487673
12440000 1.7996213001140753
12450000 1.5405778426467116
12460000 1.7569343737839656
12470000 2.8631323058838247
12480000 0.6937681637734241
12490000 0.6921083691412674
12500000 1.9962840384854077
12510000 2.0245443562196628
12520000 1.0245425360863607
12530000 1.2167251293945114
12540000 2.5283093996533754
12550000 1.801797279713737
12560000 2.5258993912304244
12570000 1.9186631427494882
12580000 1.7936203982270824
12590000 2.099248408579707
12600000 2.464907181745635
12610000 2.2561148651313054
12620000 2.709488353687465
12630000 2.747915275946756
12640000 2.7114252302928104
12650000 2.275180951579114
12660000 2.625718484829783
12670000 2.5119088609013756
12680000 2.3593865162763596
12690000 1.7109341879994204
127

15300000 1.3376467565617296
15310000 1.4392726391731498
15320000 1.700822877426644
15330000 1.1013812622629602
15340000 1.035310517605861
15350000 0.9542445463642875
15360000 0.8677448470863633
15370000 0.9098740671197374
15380000 1.1413404237974882
15390000 1.549518178975781
15400000 0.803945015160289
15410000 0.5023731789839665
15420000 0.4976315088152488
15430000 0.8522605096730722
15440000 1.2937737200106978
15450000 1.539877081832773
15460000 1.1748338476095332
15470000 1.057797755179703
15480000 0.8581190579796434
15490000 0.4154615356125898
15500000 0.2902971123740673
15510000 1.30643130774347
15520000 0.7828811574325563
15530000 1.1533613916864396
15540000 1.4433822398312224
15550000 1.4517401504992975
15560000 1.5769369360061287
15570000 0.6933337930531633
15580000 0.6092454767755666
15590000 1.6642523094852963
15600000 1.701109522521264
15610000 1.3388245027563506
15620000 0.8159125597336689
15630000 1.564011415158987
15640000 1.284893505929596
15650000 1.6393320503967403
156

18230000 0.35486771071569123
18240000 0.5613298804276057
18250000 0.7311524955020877
18260000 0.7502610736098886
18270000 0.48056721087551113
18280000 0.39774940679510434
18290000 0.37360867474828163
18300000 0.3559879601329764
18310000 0.4902453302905229
18320000 0.27512832672403253
18330000 0.42860409723550086
18340000 0.7646490765691532
18350000 0.6264732934970855
18360000 0.6973190346765386
18370000 0.25649732296184696
18380000 0.19834983964862427
18390000 0.33799196144109966
18400000 0.3442525185692906
18410000 0.4581516399291952
18420000 0.657682658502645
18430000 0.4577301854001548
18440000 0.43745768580347294
18450000 0.6709888145559298
18460000 0.3582682829067972
18470000 0.31946265299910304
18480000 0.30857792944742574
18490000 0.23347907311612368
18500000 0.1351367080192566
18510000 0.2913327027491861
18520000 0.39330797845353016
18530000 0.4279769235943556
18540000 0.41784378075783113
18550000 0.32926891504479777
18560000 0.3378887024092277
18570000 0.5037119918788208
18580